In [161]:
import codecs
import glob
import logging
import os
import pprint
import re
import csv

In [162]:
import nltk
from subprocess import Popen, PIPE
from docx import Document
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from __future__ import (
    absolute_import, division, print_function, unicode_literals
)

from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph

In [163]:
from nltk.corpus import stopwords
from nltk.tag.stanford import StanfordPOSTagger

In [164]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [165]:
os.getcwd()

'D:\\D drive\\DataAnalytics_StudyMaterial\\IIMB-BDA\\ProjectZ\\Iteration2'

In [166]:
os.chdir('D:\\D drive\\DataAnalytics_StudyMaterial\\IIMB-BDA\\ProjectZ\\Iteration2')

In [167]:
#Function to convert pdf to text
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str1 = retstr.getvalue()
    retstr.close()
    return str1

In [168]:
#Read All Docx
def iter_block_items(parent):
    """
    Generate a reference to each paragraph and table child within *parent*,
    in document order. Each returned value is an instance of either Table or
    Paragraph. *parent* would most commonly be a reference to a main
    Document object, but also works for a _Cell object, which itself can
    contain paragraphs and tables.
    """
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
        # print(parent_elm.xml)
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

In [169]:
#Function to convert doc formats (like .doc, .docx, .odt) to text
def document_to_text(filename, file_path):
    if filename[-4:] == ".doc":
        cmd = ['C:/antiword/antiword', file_path] #use antiword command
        p = Popen(cmd, stdout=PIPE)
        stdout, stderr = p.communicate()
        return stdout.decode('ascii', 'ignore')
    elif filename[-5:] == ".docx":
        doc = Document(file_path)
        fullText = []
        para_ix=0
        tbl_ix=0
        for block in iter_block_items(doc):
            block_type=block.__class__.__name__
            if block_type=='Table':
                for row in doc.tables[tbl_ix].rows:
                    for cell in row.cells:
                        for paracell in cell.paragraphs:
                            fullText.append(paracell.text)
                tbl_ix+=1
            elif block_type=='Paragraph':
                fullText.append(doc.paragraphs[para_ix].text)
                para_ix+=1
        return '\n'.join(fullText)
    elif filename[-4:] == ".odt":
        cmd = ['odt2txt', file_path]
        p = Popen(cmd, stdout=PIPE)
        stdout, stderr = p.communicate()
        return stdout.decode('ascii', 'ignore')
    elif filename[-4:] == ".pdf":
        return convert_pdf_to_txt(file_path)

In [170]:
resume_filenames = sorted(glob.glob(os.path.join('./IIM',"*.*")))

In [171]:
resume_filenames

['./IIM\\AmitBats[6_0].doc',
 './IIM\\BalakrishnaReddyS[4_6].docx',
 './IIM\\CURRENT_022017_Santosh kumartoleti_Resume_f0fb3c53-c308-4f86-b335-b05ebd2cccc9_SantoshkumarToleti2_6.docx',
 './IIM\\CURRENT_032017_BhaktiSubhash Naik_Resume_cf61cd93-ff05-4fc4-85ec-42301b5456f5_BhaktiSubhashNaik4_9.doc',
 './IIM\\CURRENT_032017_Daljeet Singh DuggalDUGGAL_Resume_adb59f9d-a2e4-46b8-abdc-375b61107dc5_DaljeetSinghDuggal6_0.docx',
 './IIM\\CURRENT_032017_DishantPatel_Resume_f88c74e3-8545-4156-a708-8d093885288e_DishantPatel6_0.docx',
 './IIM\\CURRENT_032017_HIRALPARMAR_Resume_3e8d6f3d-2b01-4603-8a09-12a840772afc_hiral2_0.pdf',
 './IIM\\CURRENT_032017_SHALINISHALINI K_Resume_e49f1c7b-d151-44b9-9afe-b774a6852495_SHALINIK2_7.docx',
 './IIM\\CURRENT_032017_SandeeV S_Resume_2cd5e682-f1f0-4d95-9483-57a25f6965c0_SandeeVS4_3.doc',
 './IIM\\CURRENT_032017_SnehlataKumari_Resume_2224bb2b-a3f9-4083-9d5d-5b0a158be3fc_SnehlataKumari2_8.doc',
 './IIM\\CURRENT_032017_ravisankar_Resume_f243d558-3e41-4fa6-a67d-f324f

In [172]:
corpus_raw = u""
for resumes in resume_filenames:
    print("Reading '{0}'...".format(resumes))
    corpus_raw+=document_to_text(resumes,os.path.join('./',resumes))
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print() 

Reading './IIM\AmitBats[6_0].doc'...
Corpus is now 14433 characters long

Reading './IIM\BalakrishnaReddyS[4_6].docx'...
Corpus is now 22915 characters long

Reading './IIM\CURRENT_022017_Santosh kumartoleti_Resume_f0fb3c53-c308-4f86-b335-b05ebd2cccc9_SantoshkumarToleti2_6.docx'...
Corpus is now 29530 characters long

Reading './IIM\CURRENT_032017_BhaktiSubhash Naik_Resume_cf61cd93-ff05-4fc4-85ec-42301b5456f5_BhaktiSubhashNaik4_9.doc'...
Corpus is now 33076 characters long

Reading './IIM\CURRENT_032017_Daljeet Singh DuggalDUGGAL_Resume_adb59f9d-a2e4-46b8-abdc-375b61107dc5_DaljeetSinghDuggal6_0.docx'...
Corpus is now 40681 characters long

Reading './IIM\CURRENT_032017_DishantPatel_Resume_f88c74e3-8545-4156-a708-8d093885288e_DishantPatel6_0.docx'...
Corpus is now 48486 characters long

Reading './IIM\CURRENT_032017_HIRALPARMAR_Resume_3e8d6f3d-2b01-4603-8a09-12a840772afc_hiral2_0.pdf'...
Corpus is now 51203 characters long

Reading './IIM\CURRENT_032017_SHALINISHALINI K_Resume_e49f1c7b-

PackageNotFoundError: Package not found at '././IIM\~$RRENT_032017_Daljeet Singh DuggalDUGGAL_Resume_adb59f9d-a2e4-46b8-abdc-375b61107dc5_DaljeetSinghDuggal6_0.docx'

In [173]:
corpus_raw

'\r\n\r\n|Professional Summary                                                      |\r\n|                                                                          |\r\n|Having 6 Years of Experience as a Software Developer in the IT industry.  |\r\n|Hands on experience in Java and J2EE (JDBC, Servlet, JSP).                |\r\n|Hands on experience in Spring Framework.                                  |\r\n|Hands on experience in Hibernate Framework.                               |\r\n|Hands on experience in Struts Framework.                                  |\r\n|Hands on experience in Tomcat, Eclipse.                                   |\r\n|Hands on experience in Design Pattern (GOF, J2EE).                        |\r\n|Hands on experience in SVN                                                |\r\n|Hands on experience in JUnit                                              |\r\n|Hands on experience in Maven,Log4J                                        |\r\n|Hands on experience in Web ser

## POS Tagging with NLTK

In [174]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(corpus_raw)

In [175]:
raw_sentences[7]

'|\r\n|Hands on experience in SVN                                                |\r\n|Hands on experience in JUnit                                              |\r\n|Hands on experience in Maven,Log4J                                        |\r\n|Hands on experience in Web service(REST)                                  |\r\n|Hands on experience in Billing Platform                                   |\r\n|Good Knowledge in Facets                                                  |\r\n|Good Knowledge in Groovy                                                  |\r\n\r\n|Employment  Summary                                                       |\r\n|                                                                                                                                   |\r\n|Programmer Analyst, Cognizant Technology Solutions, Bangalore, India                                                               |\r\n|From Dec2014-Till Date                                                     

In [176]:
st=nltk.sent_tokenize(corpus_raw)

In [177]:
wt=nltk.word_tokenize(corpus_raw)

In [178]:
print("No. Of Sentences = ",len(st))
print("No. of Word = ",len(wt))

No. Of Sentences =  2610
No. of Word =  49833


In [179]:
#pt=nltk.pos_tag(re.sub("\|","",corpus_raw).split())
pt=[]
for sent_for_tag in st:
    #tokenized_text = nltk.word_tokenize(sent_for_tag)
    #read_pos=nltk.pos_tag(tokenized_text)
    read_pos=nltk.pos_tag(re.sub("\|","",sent_for_tag).split())
    EOS_exp=[".","."]
    read_pos.append(tuple(EOS_exp))
    read_pos.append(tuple("\n"))
    pt.append(read_pos)
    #pt.append(".,.")
    
    
#pt=nltk.pos_tag(raw_sentences[0].split())

In [186]:
len(pt[1])

12

In [187]:
ctr=0
ix=0
for i in range(len(pt)):
    ctr+=len(pt[i])
print(ctr)


47102


In [216]:
#pt=[]
#sent="The Quick Brown Fox Jumped Over the Lazy Dog."# Mary Had A little Lamb. Twinkle is a little star."
#raw_sentences = tokenizer.tokenize(sent)
#for sent_for_tag in raw_sentences:
    #tokenized_text = nltk.word_tokenize(sent_for_tag)
    #read_pos=nltk.pos_tag(tokenized_text)
    #read_pos=nltk.pos_tag(re.sub("\|","",sent_for_tag).split())
    #EOS_exp=[".","."]
    #read_pos.append(tuple(EOS_exp)) 
    #read_pos.append(tuple("\n"))
    #pt.append(read_pos)

In [204]:
type(exp)
tuple(exp)

('.', '.')

## POS Tagging with Stanford NLP

In [13]:
os.environ['JAVAHOME'] = "C:/Program Files/Java/jdk1.8.0_131/bin"
CLASSPATH = "D:/D drive/Installers/stanford-postagger-full-2016-10-31/stanford-postagger.jar"
STANFORD_MODELS =  "D:/D drive/Installers/stanford-postagger-full-2016-10-31/models"


In [14]:
st_pos = StanfordPOSTagger('D:/D drive/Installers/stanford-postagger-full-2016-10-31/models/english-left3words-distsim.tagger','D:/D drive/Installers/stanford-postagger-full-2016-10-31/stanford-postagger.jar')


In [21]:
st_posTag=[]
for sent_for_tag in st:
    #tokenized_text = nltk.word_tokenize(sent_for_tag)
    #read_pos=nltk.pos_tag(tokenized_text)
    read_pos=st_pos.tag(re.sub("\|","",sent_for_tag).split())
    EOS_exp=[".","."]
    read_pos.append(tuple(EOS_exp))
    read_pos.append(tuple("\n"))
    st_posTag.append(read_pos)

#st_posTag=st_pos.tag(re.sub("\|","",corpus_raw).split())

In [22]:
st_posTag

[[('Professional', 'JJ'),
  ('Summary', 'NNP'),
  ('Having', 'VBG'),
  ('6', 'CD'),
  ('Years', 'NNS'),
  ('of', 'IN'),
  ('Experience', 'NN'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('Software', 'NNP'),
  ('Developer', 'NNP'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('IT', 'NNP'),
  ('industry.', 'NN'),
  ('.', '.'),
  ('\n',)],
 [('Hands', 'NNS'),
  ('on', 'IN'),
  ('experience', 'NN'),
  ('in', 'IN'),
  ('Java', 'NNP'),
  ('and', 'CC'),
  ('J2EE', 'NNP'),
  ('(JDBC,', 'NNP'),
  ('Servlet,', 'NNP'),
  ('JSP).', 'NNP'),
  ('.', '.'),
  ('\n',)],
 [('Hands', 'NNS'),
  ('on', 'IN'),
  ('experience', 'NN'),
  ('in', 'IN'),
  ('Spring', 'NN'),
  ('Framework.', 'NN'),
  ('.', '.'),
  ('\n',)],
 [('Hands', 'NNS'),
  ('on', 'IN'),
  ('experience', 'NN'),
  ('in', 'IN'),
  ('Hibernate', 'NNP'),
  ('Framework.', 'NNP'),
  ('.', '.'),
  ('\n',)],
 [('Hands', 'NNS'),
  ('on', 'IN'),
  ('experience', 'NN'),
  ('in', 'IN'),
  ('Struts', 'NNP'),
  ('Framework.', 'NNP'),
  ('.', '.'),
  ('\n',)],
 [('Hands', '

In [182]:
with open('POS-CV_NLTKFull.csv','w',newline='',encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['Text','POS_Tag'])
    for row in range(len(pt)):
        for col in range(len(pt[row])):
            csv_out.writerow(pt[row][col])